In [ ]:
import requests
import pandas as pd

# Hard-coded recipe IDs for specific item IDs, bug reported here: https://github.com/gw2-api/issues/issues/112
HARDCODED_RECIPES = {
    97339: 13584,  # Core tier 1
    97041: 13542,  # Core tier 2
    97284: 13550,  # Core tier 3
    96628: 13821,  # Core tier 4
    95864: 13658,  # Core tier 5
    96467: 13723,  # Core tier 6
    97020: 13751,  # Core tier 7
    96299: 13780,  # Core tier 8
    96070: 13841,  # Core tier 9
    96613: 13628,  # Core tier 10

    97487: 13839  # Piece of Dragon Jade
}

# There's no easy way to get the price of an individual research note. For now I'm asking for it explicitly. A future item would be automating this.
research_note_price = float(input("Enter the price per Research Note from https://fast.farming-community.eu/salvaging/costs-per-research-note: "))

In [184]:
class CraftingItem:

    def __init__(self, item_id):
        self.item_id = item_id
        self.item_name = None  # Item name from the API
        self.base_ingredients = {}  # Children items and their quantities
        self.raw_ingredients = None  # Raw ingredients for this item
        self.recipe_id = None  # Recipe ID for this item
        self.recipe_data = None  # Full recipe data from the API
        self.price = None  # Current buy price from the API
        self.price_instant = None  # Current instant buy price from the API
        self.crafting_cost = 0  # Total crafting cost of this item
        self.volume = None  # Volume of this item
        self.profit_margin = None  # Profit margin of this item
        self.metrics_dict = None  # Analysis metrics for this item
        # self.output_item_count = 1  # Default output count (1 if not specified)

    @staticmethod
    def api_querier(url, params=None):
        """Static method to query the Guild Wars 2 API."""
        response = requests.get(url, params=params)

        if response.status_code == 200:
            return response.json()
        elif response.json() == {"text": "no such id"}:
            return False
        else:
            raise Exception(f"API request failed with status code {response.status_code}: {response.text}")
        
    def get_item_name(self):
        """Fetch the name of this item from the API."""

        # ID 61 is for research notes, which are a different set of ids, cannot be queried from the API so hardcoding it
        if self.item_id == 61:
            self.item_name = "Research Note"
            return self.item_name

        url = f"https://api.guildwars2.com/v2/items/{self.item_id}?lang=en"
        response = self.api_querier(url)

        if response:
            self.item_name = response["name"]
            return self.item_name
        else:
            return False

    def get_recipe_id(self):
        """Fetch the recipe ID for this item."""
        # Check if the item ID has a hard-coded recipe ID
        if self.item_id in HARDCODED_RECIPES:
            self.recipe_id = HARDCODED_RECIPES[self.item_id]
            return self.recipe_id

        # Otherwise, query the API
        url = f"https://api.guildwars2.com/v2/recipes/search?output={self.item_id}"
        response = self.api_querier(url)

        if not response:
            return False
        else:
            if self.item_id == 97487:
                print(response)
            self.recipe_id = response[0]  # Store the first recipe ID
            # print(self.recipe_id)
            return self.recipe_id

    def get_recipe_data(self):
        """Fetch and store the full recipe data for this item."""
        if not self.recipe_id:
            self.get_recipe_id()  # Ensure we have a recipe ID

        if self.recipe_id:
            url = f"https://api.guildwars2.com/v2/recipes?ids={self.recipe_id}&v=latest&lang=en"
            self.recipe_data = self.api_querier(url)[0]
            self.output_item_count = self.recipe_data.get("output_item_count", 1)

            # print(self.recipe_data[0]["ingredients"])

            return self.recipe_data
        else:
            return False

    def fetch_ingredients(self):
        """
        Fetch the ingredients (children) for this item from the API.
        If the item has no recipe, it is considered a base (gatherable) item.
        """
        if not self.recipe_data:
            self.get_recipe_data()  # Ensure we have recipe data

        if self.recipe_data and "ingredients" in self.recipe_data:
            # Add children (base ingredients) to this item
            for ingredient in self.recipe_data["ingredients"]:
                # if CraftingItem(ingredient["type"]) == "Item":
                # print(ingredient)
                # print(ingredient["type"])
                if ingredient["type"] in ['Item', 'Currency']:
                    # print(ingredient["type"])
                    child_item = CraftingItem(ingredient["id"])
                    child_quantity = ingredient["count"] / self.output_item_count
                    child_item.fetch_ingredients()  # Recursively fetch children
                    self.base_ingredients[child_item] = child_quantity
        else:
            # This is a base (gatherable) item with no children
            self.base_ingredients = {}

    def get_item_price(self):
        """
        Fetch the current buy price of this item from the API.
        """
        # Query the API
        # print(self.item_id)
        if self.item_id == 61:
            self.price = research_note_price
            return self.price
            

        url = f"https://api.guildwars2.com/v2/commerce/prices/{self.item_id}"
        response = self.api_querier(url)

        if not response:
            return False
        
        # print(response)
        
        # For items not on tp, return default value (None)
        if not response["buys"]:
            return self.price
        
        
        
        else:
            self.volume = response["sells"]["quantity"] + response["buys"]["quantity"]

            self.price_instant = response["sells"]["unit_price"]

            if response["buys"]["unit_price"] == 0:
                self.price = self.price_instant
            else:
                self.price = response["buys"]["unit_price"]

        return self.price   



    def get_total_ingredients(self):
        """
        Recursively calculate the total base (gatherable) ingredients required to craft this item.
        """
        total_ingredients = {}

        for child, quantity in self.base_ingredients.items():

            # Fetch the item name for the child
            child_name = child.get_item_name()
            child_price = child.get_item_price()

            # print(total_ingredients)

            if child.base_ingredients:
                # If the child has its own ingredients, recurse
                child_ingredients = child.get_total_ingredients()
                for item_id, count_dict in child_ingredients.items():
                    if item_id in total_ingredients:
                        total_ingredients[item_id]["amount_needed"] += count_dict['amount_needed'] * quantity
                    else:
                        total_ingredients[item_id] = {
                            "name": count_dict['name'],
                            "amount_needed": count_dict['amount_needed'] * quantity,
                            "unit_price": count_dict['unit_price']
                            }
                        # total_ingredients[item_id]["amount_needed"] = count_dict * quantity
            else:
                # If the child is a base (gatherable) item, add it to the total
                if child.item_id in total_ingredients:
                    total_ingredients[child.item_id]["amount_needed"] += quantity
                else:
                    total_ingredients[child.item_id] = {
                        "name": child_name,
                        "amount_needed": quantity,
                        "unit_price": child_price
                        }

        self.raw_ingredients = total_ingredients
        return total_ingredients
    
    def calculate_crafting_cost(self):
        """
        Calculate the total crafting cost of this item.
        """
        # Fetch the total ingredients if not already done
        if not self.raw_ingredients:
            self.get_total_ingredients()

        # Calculate the total crafting cost

        for item in self.raw_ingredients:
            # print(item)
            if self.raw_ingredients[item]["unit_price"] == None:
                continue
            
            self.raw_ingredients[item]["total_cost"] = self.raw_ingredients[item]["amount_needed"] * self.raw_ingredients[item]["unit_price"]
            self.crafting_cost += self.raw_ingredients[item]["total_cost"]
            # print(self.raw_ingredients[item]["total_cost"])
            # print(self.raw_ingredients[item]["total_cost"])

        return self.crafting_cost
    
    def calculate_profit_margin(self):
        """
        Calculate the profit margin of this item.
        """
        if not self.price:
            self.get_item_price()
        
        if not self.crafting_cost:
            self.calculate_crafting_cost()

        # 15% tax on selling price
        
        self.profit_margin = self.price*.85 - self.crafting_cost
        print(f"Profit margin for {self.item_name} is {self.profit_margin/10000:.2f} gold per item")
        return self.profit_margin
    
    def get_analysis_metrics(self):
        """
        Get all analysis metrics for this item.
        """
        
        metrics_dict = {}

        metrics_dict["item_name"] = self.item_name
        metrics_dict["item_id"] = self.item_id
        metrics_dict["total_raw_resources"] = sum(item['amount_needed'] for item in self.raw_ingredients.values())
        metrics_dict["profit_margin"] = self.profit_margin
        metrics_dict["profit_per_raw"] = metrics_dict["profit_margin"] / metrics_dict["total_raw_resources"]
        metrics_dict["crafting_cost"] = self.crafting_cost
        metrics_dict["price_sell"] = self.price_instant
        metrics_dict["gap_%"] = 1 - self.price/self.price_instant
        metrics_dict["volume"] = self.volume

        self.metrics_dict = metrics_dict
        return metrics_dict

    def __repr__(self):
        return f"CraftingItem(item_id={self.item_id}, recipe_id={self.recipe_id}, base_ingredients={self.base_ingredients})"

In [185]:
cores = list(HARDCODED_RECIPES.keys())

return_dict = {}

for core in cores:
    item = CraftingItem(core)

    # Fetch the recipe data (including output_item_count)
    item.get_recipe_data()

    # Fetch the ingredients (quantities will be normalized by output_item_count)
    item.fetch_ingredients()

    # Fetch the item name
    item.get_item_name()

    # Get the total base (gatherable) ingredients required to craft one unit of the item
    total_ingredients = item.get_total_ingredients()
    # print("Total Base Ingredients (per unit):", total_ingredients)

    item.calculate_crafting_cost()

    item.calculate_profit_margin()

    return_dict[core] = item.get_analysis_metrics()

Profit margin for Jade Bot Core: Tier 1 is 0.10 gold per item
Profit margin for Jade Bot Core: Tier 2 is 0.20 gold per item
Profit margin for Jade Bot Core: Tier 3 is 0.18 gold per item
Profit margin for Jade Bot Core: Tier 4 is -0.23 gold per item
Profit margin for Jade Bot Core: Tier 5 is -0.58 gold per item
Profit margin for Jade Bot Core: Tier 6 is 1.23 gold per item
Profit margin for Jade Bot Core: Tier 7 is 2.15 gold per item
Profit margin for Jade Bot Core: Tier 8 is 2.32 gold per item
Profit margin for Jade Bot Core: Tier 9 is 1.42 gold per item
Profit margin for Jade Bot Core: Tier 10 is 3.64 gold per item
Profit margin for Piece of Dragon Jade is -0.06 gold per item


In [186]:
return_dict

{97339: {'item_name': 'Jade Bot Core: Tier 1',
  'item_id': 97339,
  'total_raw_resources': 60.0,
  'profit_margin': 1032.699999999999,
  'profit_per_raw': 17.21166666666665,
  'crafting_cost': 12331.0,
  'price_sell': 23396,
  'gap_%': 0.32800478714310144,
  'volume': 1611},
 97041: {'item_name': 'Jade Bot Core: Tier 2',
  'item_id': 97041,
  'total_raw_resources': 136.0,
  'profit_margin': 1976.5999999999985,
  'profit_per_raw': 14.533823529411753,
  'crafting_cost': 25713.0,
  'price_sell': 49502,
  'gap_%': 0.34192557876449436,
  'volume': 598},
 97284: {'item_name': 'Jade Bot Core: Tier 3',
  'item_id': 97284,
  'total_raw_resources': 277.0,
  'profit_margin': 1808.4000000000015,
  'profit_per_raw': 6.528519855595673,
  'crafting_cost': 47070.0,
  'price_sell': 84704,
  'gap_%': 0.3211182470721572,
  'volume': 242},
 96628: {'item_name': 'Jade Bot Core: Tier 4',
  'item_id': 96628,
  'total_raw_resources': 403.0,
  'profit_margin': -2303.25,
  'profit_per_raw': -5.715260545905707,

In [187]:
df = pd.DataFrame.from_dict(return_dict, orient='index')

df.head(11)

,item_name,item_id,total_raw_resources,profit_margin,profit_per_raw,crafting_cost,price_sell,gap_%,volume
97339,Jade Bot Core: Tier 1,97339,60.0,1032.70,17.211667,12331.0,23396,0.328005,1611
97041,Jade Bot Core: Tier 2,97041,136.0,1976.60,14.533824,25713.0,49502,0.341926,598
97284,Jade Bot Core: Tier 3,97284,277.0,1808.40,6.528520,47070.0,84704,0.321118,242
96628,Jade Bot Core: Tier 4,96628,403.0,-2303.25,-5.715261,62122.0,124403,0.434298,177
95864,Jade Bot Core: Tier 5,95864,599.0,-5819.15,-9.714775,86604.0,159799,0.405247,159
96467,Jade Bot Core: Tier 6,96467,775.0,12252.80,15.810065,107604.0,195799,0.279833,207
97020,Jade Bot Core: Tier 7,97020,1001.0,21534.40,21.512887,138354.0,268807,0.300227,116
96299,Jade Bot Core: Tier 8,96299,1357.0,23186.10,17.086293,172914.0,319500,0.277915,111
96070,Jade Bot Core: Tier 9,96070,1773.0,14217.20,8.018725,221804.0,398400,0.303032,150
96613,Jade Bot Core: Tier 10,96613,2626.0,36365.35,13.848191,310665.0,449988,0.092707,488
